In [1]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\data_.csv")
df

In [ ]:
df.info()

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['day0'] = pd.to_datetime(df['day0'])
df.info()

In [ ]:
df.head(2)

In [ ]:
new_df = df
new_df.head(2)

In [ ]:
new_df['version'].unique()

# Guide

In [ ]:
# New players (day_diff = 0), playing at level 1
new_user = new_df[(new_df['day_diff'] == 0) & (new_df['event_name'] == 'tutorial') & (new_df['level'] == 1)]
new_user

## Repeat view guide

In [ ]:
# qty = -1: start entering guide
times = new_user[new_user['quantity'] == -1].groupby(['version', 'user']).agg(times_view_guide = ('STT', 'count')).reset_index()
times

## Finish step or not

In [ ]:
# qty = -2: completed all guide steps
# qty = 0: skip guide
finish = new_user[new_user['quantity'].isin([-2, 0])][['user', 'quantity', 'version']].drop_duplicates()
finish['is_finish'] = finish.quantity.apply(lambda x: 0 if x == 0 else 1)
finish

## Final table

In [ ]:
final = pd.merge(times,
         finish,
         how = 'inner',
         on = ['user', 'version'])

final

## Z test

### completed

Is there a difference between the completion rates of the tutorial in the old version and the new version?

In [ ]:
finish_group = final.groupby('version').agg(total_user = ('user', 'nunique'),
                             user_completed = ('is_finish', 'sum')).reset_index()

finish_group['completion_rate'] = finish_group['user_completed'] / finish_group['total_user']

finish_group

In [72]:
count = finish_group['user_completed'].values
nobs = finish_group['total_user'].values

z_stat, p_value = proportions_ztest(count, nobs)
print(f'Z = {z_stat:.2f}, P-value = {p_value:.4f}')

if p_value < 0.05:
  print('There is a difference between the 2 versions')
else:
  print('There is no difference between the 2 versions')

Z = -0.56, P-value = 0.5733
There is no difference between the 2 versions


### reguide

Is there a difference in the review ratings of the instructions between the old version and the new version?

- H0: review ratings verB (yes) = verA
- H1: review ratings verB (yes) <> verA 

In [ ]:
reguide = pd.read_csv(r"C:\Users\Admin\Downloads\reguide.csv")
reguide

In [ ]:
reguide.groupby('version').sum()

In [67]:
counts = [103, 100]
nobs = [2435, 2846]

In [73]:
stat, pval = proportions_ztest(counts, nobs, alternative = 'two-sided')
print(f'Z = {stat:.2f}, P-value = {pval:.4f}')

if p_value < 0.05:
  print('There is a difference between the 2 versions')
else:
  print('There is no difference between the 2 versions')

Z = 0.46, P-value = 0.6488
There is no difference between the 2 versions


### churn

Is there a difference between the churn rates of the old version and the new version?

- H0: churn rate verB >= verA
- H1: churn rate verB < verA

In [ ]:
churn = pd.read_csv(r"C:\Users\Admin\Downloads\churn.csv")

churn

In [90]:
churn = [4230, 4087]
total = [6669, 6927]

In [91]:
stat, pval = proportions_ztest(churn, total, alternative = 'larger')
print("Z-stat =", stat, "p-value =", pval)

if pval < 0.05:
  print('The churn rate of the new version is lower.')
else:
  print('The churn rate of the new version is higher.')

Z-stat = 5.294662020967035 p-value = 5.961836577265189e-08
The churn rate of the new version is lower.


### live plus

Is there a difference between the used live plus rates of the old version and the new version?

- H0: use live plus rate verB <= verA
- H1: use live plus rate verB > verA

In [ ]:
live = pd.read_csv(r"C:\Users\Admin\Downloads\live.csv")
live

In [92]:
counts = [839, 986]
nobs = [5182, 5532]

In [93]:
stat, pval_two_sided = proportions_ztest(counts, nobs, alternative='two-sided')
print("p-value (two sided) =", pval_two_sided)


p-value (two sided) = 0.02465080052895374


In [95]:
stat, pval_one_sided = proportions_ztest(counts, nobs, alternative='smaller')
print("Z-stat =", stat)
print("p-value (one sided, 1.6.0 > 1.5.2) =", pval_one_sided)

if pval < 0.05:
  print('The used live plus of the new version is higher')
else:
  print('The used live plus of the new version is lower')

Z-stat = -2.2468314574336454
p-value (one sided, 1.6.0 > 1.5.2) = 0.01232540026447687
The used live plus of the new version is higher
